Avani Agarwal

102303745

Assignment-3


Q1.

(a)Divide dataset into input features (X) and output variable (y)

(b)Scale the input features

(c)5-Fold Cross Validation

(d)Iterate over folds, perform training and testing

(e)Train on 70% data and test on 30% with best beta

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

url = 'https://drive.google.com/uc?id=1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX'
df = pd.read_csv(url)

print(df.columns)

#(a)
X = df.drop(columns=['Avg. Area House Age'])
y = df['Avg. Area House Age']


#(b)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#(c)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
best_beta = None
best_r2_score = -np.inf

#(d)
for train_index, test_index in kf.split(X_scaled):
   X_train, X_test = X_scaled[train_index], X_scaled[test_index]
   y_train, y_test = y.iloc[train_index], y.iloc[test_index]

   # Compute beta (𝛽) using least squares method
   beta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
   y_pred = X_test @ beta
   r2 = r2_score(y_test, y_pred)

   # Save beta if it gives the best R2 score
   if r2 > best_r2_score:
       best_r2_score = r2
       best_beta = beta

#(e)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)
y_pred_final = X_test @ best_beta
final_r2_score = r2_score(y_test, y_pred_final)


print("Best Beta (𝛽) Matrix:\n", best_beta)
print("Final R² Score on 30% test data:", final_r2_score)

Index(['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population', 'Price'],
      dtype='object')
Best Beta (𝛽) Matrix:
 [-0.99470426 -0.52636639  0.00233564 -0.67156612  1.47168908]
Final R² Score on 30% test data: -36.575580506937904


Q2.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

url = 'https://drive.google.com/uc?id=1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX'
df = pd.read_csv(url)

X = df.drop(columns=['Avg. Area House Age'])
y = df['Avg. Area House Age']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
   X_scaled, y, test_size=0.3, random_state=1
)
X_train, X_val, y_train, y_val = train_test_split(
   X_train_val, y_train_val, test_size=0.2, random_state=1  # 14% validation set
)

#Define Gradient Descent function
def gradient_descent(X, y, learning_rate, iterations):
   n_features = X.shape[1]
   beta = np.zeros(n_features)
   for _ in range(iterations):
       y_pred = X @ beta
       gradient = (1 / len(y)) * X.T @ (y_pred - y)
       beta = beta - learning_rate * gradient
   return beta

#Iterate over learning rates and compute regression coefficients
learning_rates = [0.001, 0.01, 0.1, 1]
best_beta = None
best_r2_val = -np.inf

for lr in learning_rates:
   beta = gradient_descent(X_train, y_train, lr, iterations=1000)
   y_pred_val = X_val @ beta
   r2_val = r2_score(y_val, y_pred_val)

   print(f"Learning Rate: {lr}, Validation R2 Score: {r2_val}")

   if r2_val > best_r2_val:
       best_r2_val = r2_val
       best_beta = beta


y_pred_test = X_test @ best_beta
final_r2_score = r2_score(y_test, y_pred_test)

print("\nBest Beta (𝛽) Matrix:\n", best_beta)
print("Final R² Score on 30% test data:", final_r2_score)

Learning Rate: 0.001, Validation R2 Score: -38.132257859885776
Learning Rate: 0.01, Validation R2 Score: -37.570121443792516
Learning Rate: 0.1, Validation R2 Score: -37.507451918915464
Learning Rate: 1, Validation R2 Score: -37.50745190594605

Best Beta (𝛽) Matrix:
 [-1.02283309 -0.69932608 -0.06173613 -0.69519406  1.69563527]
Final R² Score on 30% test data: -36.70071148752416


Q3.

1. Load the dataset and replace '?' with NaN

2. Replace NaN values with central tendency imputation and drop rows with NaN in 'price'

3. Convert non-numeric values to numeric

(i) num_doors and num_cylinders

(ii) body_style, drive_wheels, engine_location, engine_type, fuel_system, make, aspiration, fuel_type

4. Split data, train model, and evaluate

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# 1.
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
column_names = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration", "num_doors", "body_style",
               "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight",
               "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
               "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]
df = pd.read_csv(url, header=None, names=column_names, na_values=['?'])

# 2.
for col in df.columns:
   if df[col].dtype == 'object':  # Impute categorical features with mode
       df[col] = df[col].fillna(df[col].mode()[0])
   else:  # Impute numerical features with mean
       df[col] = df[col].fillna(df[col].mean())

df.dropna(subset=['price'], inplace=True)

# 3.
# (i)
num_word_mapping = {'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'eight': 8, 'twelve': 12}
df['num_doors'] = df['num_doors'].map(num_word_mapping)
df['num_cylinders'] = df['num_cylinders'].map(num_word_mapping)

# (ii)
categorical_cols = ["body_style", "drive_wheels", "engine_location", "engine_type", "fuel_system", "make", "aspiration", "fuel_type"]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 4.
X = df.drop(columns=['price'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print("R-squared:", r2)

R-squared: 0.8813554308249376
